# MongoDB Queryable Encryption with Atlas - Developer Education

This notebook is based on the available  [Queryable Encryption quick start](https://www.mongodb.com/docs/manual/core/queryable-encryption/quick-start/).

# Installation of prerequisites

In [1]:
!pip install pymongo[encryption]
!pip install pymongo
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [2]:

import os
from pymongo import MongoClient
from pymongo.encryption import Algorithm, ClientEncryption, QueryType
from pymongo.encryption_options import AutoEncryptionOpts
from bson.codec_options import CodecOptions
from bson import json_util
import json
import requests
import platform
import tempfile

Function to download and set up crypt_shared library

In [3]:
def setup_crypt_shared():
    system = platform.system().lower()
    if system == "linux":
        url = "https://downloads.mongodb.com/linux/mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"
        filename = "mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"
    elif system == "darwin":
        url = "https://downloads.mongodb.com/osx/mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"
        filename = "mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"
    elif system == "windows":
        url = "https://downloads.mongodb.com/windows/mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"
        filename = "mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"
    else:
        raise OSError("Unsupported operating system")

    response = requests.get(url)
    response.raise_for_status()

    with tempfile.NamedTemporaryFile(delete=False, suffix=".tgz" if system != "windows" else ".zip") as tmp_file:
        tmp_file.write(response.content)
        tmp_file_path = tmp_file.name

    extract_dir = tempfile.mkdtemp()

    if system != "windows":
        os.system(f"tar -xzf {tmp_file_path} -C {extract_dir}")
        lib_path = os.path.join(extract_dir, "lib", "mongo_crypt_v1.so")
    else:
        os.system(f"powershell Expand-Archive -Path {tmp_file_path} -DestinationPath {extract_dir}")
        lib_path = os.path.join(extract_dir, "bin", "mongo_crypt_v1.dll")

    return lib_path

# Set up crypt_shared library

In [4]:
crypt_shared_lib_path = setup_crypt_shared()
print(f"Crypt shared library path: {crypt_shared_lib_path}")


Crypt shared library path: /tmp/tmpsyx58qew/lib/mongo_crypt_v1.so


 1. Connect to your Atlas cluster

In [49]:
import getpass
MONGODB_ATLAS_URI = getpass.getpass("Enter your MongoDB Atlas URI:")

key_vault_client = MongoClient(MONGODB_ATLAS_URI, appname="deverel.content.python")

Enter your MongoDB Atlas URI:··········


2. Set up encryption key and providers

In [41]:
local_master_key = os.urandom(96)
kms_providers = {"local": {"key": local_master_key}}
key_vault_namespace = "encryption.__keyVault"


In [42]:
kms_provider_name="local"
key_vault_database_name = "encryption"
key_vault_collection_name = "__keyVault"
key_vault_namespace = f"{key_vault_database_name}.{key_vault_collection_name}"
encrypted_database_name = "medicalRecords"
encrypted_collection_name = "patients"

3. Create a ClientEncryption object

In [43]:
auto_encryption_options = AutoEncryptionOpts(
    kms_providers,
    key_vault_namespace,
    crypt_shared_lib_path=crypt_shared_lib_path
)

 4. Set up the key vault

In [50]:
encrypted_client = MongoClient(
    MONGODB_ATLAS_URI, auto_encryption_opts=auto_encryption_options)


6. Define the encrypted fields map

In [54]:
encrypted_fields_map = {
    "fields": [
        {
            "path": "patientRecord.ssn",
            "bsonType": "string",
            "queries": [{"queryType": "equality"}] # potentially 'range'
        },
        {
            "path": "patientRecord.billing",
            "bsonType": "object",
        }
    ]
}

7. Create an encrypted client

In [52]:
client_encryption = ClientEncryption(
    kms_providers=kms_providers,
    key_vault_namespace=key_vault_namespace,
    key_vault_client=encrypted_client,
    codec_options=CodecOptions()
)




 8. Set up and use the encrypted collection

In [55]:
client_encryption.create_encrypted_collection(
    encrypted_client[encrypted_database_name],
    encrypted_collection_name,
    encrypted_fields_map,
    kms_provider_name,
    {},
)

(Collection(Database(MongoClient(host=['serverless1-lb.uvwhr.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', loadbalanced=True, tls=True, auto_encryption_opts=<pymongo.encryption_options.AutoEncryptionOpts object at 0x7f57846ba980>), 'medicalRecords'), 'patients'),
 {'fields': [{'path': 'patientRecord.ssn',
    'bsonType': 'string',
    'queries': [{'queryType': 'equality'}],
    'keyId': Binary(b'L\x04.\x94\x91\xaaL\x10\x8fb\xe1\x91+\xaeg\x8b', 4)},
   {'path': 'patientRecord.billing',
    'bsonType': 'object',
    'keyId': Binary(b'YV\x0cP\x1f\xfeF \x96".V:\xec\x82\xf5', 4)}]})

9. Insert an encrypted document

In [56]:
patient_document = {
    "patientName": "Jon Doe",
    "patientId": 12345678,
    "patientRecord": {
        "ssn": "987-65-4320",
        "billing": {
            "type": "Visa",
            "number": "4111111111111111",
        },
    },
}
encrypted_collection = encrypted_client[encrypted_database_name][encrypted_collection_name]
result = encrypted_collection.insert_one(patient_document)
print(f"Inserted document ID: {result.inserted_id}")

Inserted document ID: 66aa6a8fc652e5247e1cbc7f


10. Query the encrypted collection

In [57]:
find_result = encrypted_collection.find_one({
    "patientRecord.ssn": "987-65-4320"
})
print(find_result)


{'_id': ObjectId('66aa6a8fc652e5247e1cbc7f'), 'patientName': 'Jon Doe', 'patientId': 12345678, 'patientRecord': {'ssn': '987-65-4320', 'billing': {'type': 'Visa', 'number': '4111111111111111'}}, '__safeContent__': [b'\xd7\xb5&\xd7I\x9eF;g\xb8\x9dEmGkbd}\x8c\xf7w\x87\xb1Y(\xa3b\xbf\x1e\x9fW\x01']}


In [58]:
print("\nQuerying without encryption:")

reg_mongoclient = MongoClient(MONGODB_ATLAS_URI)

find_result = reg_mongoclient[encrypted_database_name][encrypted_collection_name].find_one({
    "patientRecord.ssn": "987-65-4320"
})
print(find_result)

all_docs = reg_mongoclient[encrypted_database_name][encrypted_collection_name].find()
print("\nAll documents in the collection:")
for doc in all_docs:
    print(doc)


Querying without encryption:
None

All documents in the collection:
{'_id': ObjectId('66aa6a8fc652e5247e1cbc7f'), 'patientName': 'Jon Doe', 'patientId': 12345678, 'patientRecord': {'ssn': Binary(b'\x0eL\x04.\x94\x91\xaaL\x10\x8fb\xe1\x91+\xaeg\x8b\x02k13\xecIk\x90\x81\x15\xbf\x08i\xa4z\x10\xfc\x11O!\xc9p\xce\x12\x0c\xd3\xa9\xb1(\xed\x90\xb2\x91q\xc2\xab\xc6\xb4\x827C\xf0\xfd\xc4\xf9\x88W<11_\xa3\x98\x18\x1b\xba)\xb6\x12%)4\n|\x1f\xea\x00\x12(\x8f@\x1a0\xd1`\xfa\x82/A2\xdf9\xe3\xb3\x8d$\t\xba|\x9fkDfsf\xa5\x08\xbb\x1en%\x11\xb1\xd0s\x87k\x92+j\x85\xd6\xdf;K\x88\x1e\xf0S>Z_G@\x7fU+\xb6\x93\x18<#bmJ\xd6\xba\x86F}r\xacM%\xee\xd7\xb5&\xd7I\x9eF;g\xb8\x9dEmGkbd}\x8c\xf7w\x87\xb1Y(\xa3b\xbf\x1e\x9fW\x01\xba\x9c#G\x9f\xcd\x82\xf2\x8d\x1b`\xf6\x06\x17pi\x0b\xc8\xde\x90\xc3}Q\x19\xcd\r<=a\x15\x03\xce', 6), 'billing': Binary(b'\x10YV\x0cP\x1f\xfeF \x96".V:\xec\x82\xf5\x03<\x12\xcdT\x10\xad}\xbb\x8a\xd2\xa1d\xaf&\xee\xcf\x05\x91\xe8w\xa6\x1bs\xa7\xe1\xcf\x83(\xb0;(\xe5\xc5\x0ec\xc7y\xd02\x88+M\xb